In [1]:
import pandas as pd
from fuzzywuzzy import process

# Define the acceptable values
resource_class_values = ['Collections','Datasets','Imagery','Maps','Web services','Websites','Other']
access_rights_values = ['Public', 'Restricted']

# Load your CSV file into a pandas DataFrame
csv_file_path = 'arc.csv'
data = pd.read_csv(csv_file_path)

# Create a DataFrame to store cleaning log
cleaning_log = pd.DataFrame(columns=['ColumnName', 'OriginalValue', 'CleanedValue', 'CleaningAction'])

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
# Clean 'Resource Class'
def clean_resource_class(row):
    resource_class_string = row['Resource Class']
    original = resource_class_string
    if pd.isnull(resource_class_string):
        new_value = 'Datasets'
        global cleaning_log
        cleaning_log = pd.concat([cleaning_log, pd.DataFrame([{ 'ID': row['ID'], 'ColumnName': 'Resource Class', 'OriginalValue': original, 'CleanedValue': new_value, 'CleaningAction': 'Filled empty value with "Datasets"'}])], ignore_index=True)
        return new_value
    else:
        resource_classes = resource_class_string.split('|')
        new_resource_classes = []
        for class_value in resource_classes:
            new_value = process.extractOne(class_value.strip(), resource_class_values)[0]
            if new_value != class_value:
                cleaning_log = pd.concat([cleaning_log, pd.DataFrame([{ 'ID': row['ID'], 'ColumnName': 'Resource Class', 'OriginalValue': class_value, 'CleanedValue': new_value, 'CleaningAction': 'Fixed spelling'}])], ignore_index=True)
            new_resource_classes.append(new_value)
        return '|'.join(new_resource_classes)
data['Resource Class'] = data.apply(clean_resource_class, axis=1)

In [3]:
# Clean 'Date Range'
def clean_date_range(row):
    x = row['Date Range']
    original = x
    if pd.isnull(x) or x == '':
        return x  # returns the original value if it's empty or null
    else:
        date_ranges = str(x).split('|')
        for i in range(len(date_ranges)):
            years = date_ranges[i].split('-')
            if len(years) != 2 or not years[0].isdigit() or not years[1].isdigit() or int(years[0]) > int(years[1]):
                years = sorted(years)
                date_ranges[i] = '-'.join(years)
                x = '|'.join(date_ranges)
                global cleaning_log
                cleaning_log = pd.concat([cleaning_log, pd.DataFrame([{ 'ID': row['ID'], 'ColumnName': 'Date Range', 'OriginalValue': original, 'CleanedValue': x, 'CleaningAction': 'Corrected date order'}])], ignore_index=True)
        return x
data['Date Range'] = data.apply(clean_date_range, axis=1)

In [4]:
# Clean 'Access Rights'
def clean_access_rights(row):
    x = row['Access Rights']
    original = x
    if pd.isnull(x):
        x = 'Public'
        global cleaning_log
        cleaning_log = pd.concat([cleaning_log, pd.DataFrame([{ 'ID': row['ID'], 'ColumnName': 'Access Rights', 'OriginalValue': original, 'CleanedValue': x, 'CleaningAction': 'Filled empty value with "Public"'}])], ignore_index=True)
    else:
        new_value = process.extractOne(str(x), access_rights_values)[0]
        if new_value != x:
            x = new_value
            cleaning_log = pd.concat([cleaning_log, pd.DataFrame([{ 'ID': row['ID'], 'ColumnName': 'Access Rights', 'OriginalValue': original, 'CleanedValue': x, 'CleaningAction': 'Fixed spelling'}])], ignore_index=True)
    return x

data['Access Rights'] = data.apply(clean_access_rights, axis=1)

In [5]:
# Clean 'Format' based on 'Download' field
def clean_format(row):
    x = row['Format']
    original = x
    if pd.isnull(x) and pd.notnull(row['Download']):
        x = 'File'
        global cleaning_log
        cleaning_log = pd.concat([cleaning_log, pd.DataFrame([{ 'ID': row['ID'], 'ColumnName': 'Format', 'OriginalValue': original, 'CleanedValue': x, 'CleaningAction': 'Filled missing value with "File"'}])], ignore_index=True)
    return x

data['Format'] = data.apply(clean_format, axis=1)

In [6]:
# Function to round decimal places
def round_coordinates(row):
    x = row['Bounding Box']
    original = x
    if pd.isnull(x) or x == '':
        return x
    else:
        pairs = x.split(',')
        new_pairs = []
        for pair in pairs:
            coords = pair.split()
            new_coords = [str(round(float(coord), 2)) for coord in coords]
            new_pair = ' '.join(new_coords)
            global cleaning_log
            if new_pair != pair:
                cleaning_log = pd.concat([cleaning_log, pd.DataFrame([{'ID':row['ID'], 'ColumnName': 'Bounding Box', 'OriginalValue': pair, 'CleanedValue': new_pair, 'CleaningAction': 'Rounded to 2 decimal places'}])], ignore_index=True)
            new_pairs.append(new_pair)
        return ','.join(new_pairs)

data['Rounded Bounding Box'] = data.apply(round_coordinates, axis=1)

In [ ]:
# Function to check and correct points/lines

def correct_points_lines(row):
    x = str(row['Rounded Bounding Box'])
    original = x
    if pd.isnull(x) or x == '':
        return x
    else:
        pairs = x.split(',')
        new_pairs = []
        for pair in pairs:
            coords = pair.split()
            if len(coords) == 4:
                west, south, east, north = map(float, coords)
                if west == east:
                    east = round(east + 0.01, 2)
                if south == north:
                    north = round(north + 0.01, 2)
                new_pair = ' '.join(map(str, [west, south, east, north]))
                if new_pair != pair:
                    global cleaning_log
                    cleaning_log = pd.concat([cleaning_log, pd.DataFrame([{'ID':row['ID'], 'ColumnName': 'Bounding Box', 'OriginalValue': pair, 'CleanedValue': new_pair, 'CleaningAction': 'Corrected line/point to a box'}])], ignore_index=True)
                new_pairs.append(new_pair)
        return ','.join(new_pairs)

data['Cleaned Bounding Box'] = data.apply(correct_points_lines, axis=1)

In [ ]:
# After cleaning
cleaned_file_path = "cleaned_metadata_2.csv"
data.to_csv(cleaned_file_path, index=False)
cleaning_log.to_csv("cleaning_log_2.csv", index=False)